In [1]:
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch
from huggingface_hub import login
from dotenv import load_dotenv
import os

In [2]:
# Load the environment file
load_dotenv("token.env")

# Retrieve the API token
api_token = os.getenv("api_token")

login(token = api_token)

In [3]:
model_kwargs = {
    'do_sample' : True,
    'temperature': 0,
    'max_length' : 1000
}

pipeline_kwargs = {
    'max_new_tokens' : 512
}

In [4]:
model_id = 'google/gemma-7b-it'

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_quant_type = 'nf4',
#     bnb_4bit_compute_dtype = torch.float16
# )

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype = torch.bfloat16 , trust_remote_code = True, device_map = 'cuda')
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=100)
hf = HuggingFacePipeline(pipeline=pipe, model_kwargs = model_kwargs, pipeline_kwargs = pipeline_kwargs)

In [15]:
pipe('What is this')

[{'generated_text': 'What is this'}]

In [14]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables = ['question'])

llm_chain = LLMChain(prompt = prompt, llm = hf, verbose = True)

question = "What is electroencephalography?"

answer = llm_chain.invoke({"question": question})



> Entering new LLMChain chain...
Prompt after formatting:
Question: What is electroencephalography?

Answer: Let's think step by step.

> Finished chain.


In [ ]:
template = """
You are a friendly chatbot assistant that responds conversationally to users' questions.
Keep the answers short, unless specifically asked by the user to elaborate on something.

Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
def ask_question(question):
    result = llm_chain(question)
    print(result['question'])
    print("")
    print(result['text'])

In [ ]:
ask_question('Describe yourself')

In [27]:
pipe = pipeline("text-generation", 
               model=model, 
               tokenizer=tokenizer, 
               max_new_tokens=512,
               do_sample=True,
               temperature=1,
               )
hf = HuggingFacePipeline(pipeline=pipe)

query = "Who is Shah Rukh Khan?"

hf = HuggingFacePipeline(pipeline=pipe)

query = "describe youself"

prompt = f"""
### System:
You are an AI assistant that follows instruction extremely well. 
Help as much as you can. Please be truthful and give direct answers

### User:
{query}

### Response:
"""

response = hf.predict(prompt)
print(response)

In [10]:
template = """
You are a friendly chatbot assistant that responds conversationally to users' questions.
Keep the answers short, unless specifically asked by the user to elaborate on something.

Question: {question}

Answer:"""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=hf)

In [18]:
tokeni

AttributeError: 'str' object has no attribute 'shape'

In [14]:
def ask_question(question):
    result = llm_chain.invoke(question)
    print(result['question'])
    print("")
    print(result['text'])

In [12]:
import time

class TimerError(Exception):
    ''' A custom excpetion used to report errors in use of Timer Class'''

class Timer:
    def __init__(self):
        self._start_time = None
    
    def __enter__(self):
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")
        self._start_time = time.perf_counter()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")
        elapsed_time = time.perf_counter() - self._start_time
        print(f'Elapsed Time: {elapsed_time:0.4f} seconds')

In [15]:
with Timer():
    ask_question("Describe some famous landmarks in London")

Describe some famous landmarks in London

 Describe landmarks
Elapsed Time: 1.6967 seconds


In [31]:
ask_question('Describe some famous anime characters')

Describe some famous anime characters
****************************************************************************************************
 Describe some famous anime characters
Elapsed Time: 0.4790 seconds
